# 1. Install AG2 + Materials Project dependencies

In [1]:
!pip install -U "ag2[openai]" autogen -q

import sys
!{sys.executable} -m pip install pymatgen mp-api numpy cython ipywidgets jupyterlab_widgets -q

print("✓ AG2/autogen + Pymatgen + MP-API installed.")

✓ AG2/autogen + Pymatgen + MP-API installed.


# 2. Imports + LLM Configuration


In [2]:
import os
import json
import random
from datetime import datetime, timedelta
from typing import Annotated, Any, Literal, Union, List, Optional, Tuple

from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import ReplyResult
from autogen.agentchat.group import (
    ContextVariables,
    AgentTarget, AgentNameTarget, StayTarget,
    OnCondition, StringLLMCondition,
    OnContextCondition, ExpressionContextCondition, ContextExpression,
    RevertToUserTarget, TerminateTarget
)
from autogen.agentchat.groupchat import GroupChat, GroupChatManager
from autogen.tools import tool

from mp_api.client import MPRester

from autogen.coding.local_commandline_code_executor import LocalCommandLineCodeExecutor
from autogen.coding.base import CodeBlock
from pydantic import BaseModel, Field

from autogen.agentchat.group.patterns import DefaultPattern
from autogen.agentchat import initiate_group_chat

# LLM configuration
llm_config = LLMConfig(
    config_list=[
        {
            "model": "gpt-4o",
            "api_key": os.environ["OPENAI_API_KEY"],
        }
    ],
)
print("✓ Imports loaded and LLM configured.")

✓ Imports loaded and LLM configured.


# 3. User query + Context Variables

In [3]:
from autogen.agentchat.group import ContextVariables

context_variables = ContextVariables(
    data={
        # Task lifecycle
        "task_started": False,
        "query": None,

        # Task refinement
        "main_task_analysis": None,
        "main_task_improvements": None,
        "main_task_improved": None,

        # Planning
        "plan": None,
        "review_status": None,
        "review_notes": None,

        # Retrieval
        "search_criteria": None,
        "fields": None,
        "sample_number": None,
        "mp_results": None,

        # Analysis
        "final_conclusion": None,

        # Coding outputs
        "last_executed_code": None,
        "last_execution_output": None,
        "last_executed_file": None,

        # Routing
        "next_agent": None,
    }
)

print("✓ ContextVariables initialized and aligned with AG2 pipeline.")


✓ ContextVariables initialized and aligned with AG2 pipeline.


# 4. TOOLS — TaskImprover → Planner → PlanReviewer → MaterialsRetriever → Analyzer → Coder


In [4]:
# Tool — task_improver

def task_improver_tool(
    analysis: Annotated[str, "Short analysis of the task consistency, missing info, redundancy."],
    improvements: Annotated[str, "Concrete improvements to make the task complete and non-redundant."],
    improved_task: Annotated[str, "Improved version of the main task."],
    context_variables: ContextVariables,
) -> ReplyResult:
    """Improve the provided main task for the specific goal it sets."""

    agent_name = "AgentTaskImprover"

    try:
        if not isinstance(analysis, str) or not analysis.strip():
            raise ValueError("analysis must be a non-empty string.")
        if not isinstance(improvements, str) or not improvements.strip():
            raise ValueError("improvements must be a non-empty string.")
        if not isinstance(improved_task, str) or not improved_task.strip():
            raise ValueError("improved_task must be a non-empty string.")
    except ValueError as e:
        context_variables["next_agent"] = agent_name
        return ReplyResult(
            message=f"Error in task_improver_tool: {e}",
            target=AgentNameTarget(agent_name),
            context_variables=context_variables,
        )

    analysis_s = analysis.strip()
    improvements_s = improvements.strip()
    improved_task_s = improved_task.strip()

    context_variables["task_started"] = True
    context_variables["main_task_analysis"] = analysis_s
    context_variables["main_task_improvements"] = improvements_s
    context_variables["main_task_improved"] = improved_task_s
    context_variables["next_agent"] = "AgentPlanner"

    message = {
        "main_task_analysis": analysis_s,
        "main_task_improvements": improvements_s,
        "main_task_improved": improved_task_s,
    }

    return ReplyResult(
        message=json.dumps(message, indent=2),
        target=AgentNameTarget("AgentPlanner"),
        context_variables=context_variables,
    )

# Tool — planner

def planner_tool(
    context_variables: ContextVariables,
) -> ReplyResult:
    agent_name = "AgentPlanner"
    main_task_improved = context_variables.get("main_task_improved", None)

    try:
        if not isinstance(main_task_improved, str) or not main_task_improved.strip():
            raise ValueError("context_variables['main_task_improved'] must be a non-empty string.")
    except ValueError as e:
        context_variables["next_agent"] = agent_name
        return ReplyResult(
            message=f"Error in planner_tool: {e}",
            target=AgentNameTarget(agent_name),
            context_variables=context_variables,
        )

    main_task_s = main_task_improved.strip()

    guardrails = {
        "sample_number": {"default": 25, "min": 1, "max": 200},
        "notes": [
            "Retriever must use only whitelisted Materials Project keys/fields.",
            "If query is ambiguous, prefer conservative filters and keep the plan iterative.",
            "Do not claim properties not present in retrieved fields.",
        ],
    }

    sn = guardrails.get("sample_number", {})
    if not (
        isinstance(sn, dict)
        and isinstance(sn.get("default"), int)
        and isinstance(sn.get("min"), int)
        and isinstance(sn.get("max"), int)
    ):
        context_variables["next_agent"] = agent_name
        return ReplyResult(
            message="Error in planner_tool: guardrails['sample_number'] must have int keys: default/min/max.",
            target=AgentNameTarget(agent_name),
            context_variables=context_variables,
        )
    if sn["min"] <= 0 or sn["max"] < sn["min"] or not (sn["min"] <= sn["default"] <= sn["max"]):
        context_variables["next_agent"] = agent_name
        return ReplyResult(
            message="Error in planner_tool: invalid sample_number guardrails (min/max/default).",
            target=AgentNameTarget(agent_name),
            context_variables=context_variables,
        )

    defaults = {
        "sample_number": sn["default"],
        "fields_minimum": [
            "material_id",
            "formula_pretty",
            "band_gap",
            "energy_above_hull",
            "density",
            "volume",
            "symmetry",
            "nsites",
            "elements",
            "chemsys",
            "is_stable",
        ],
    }

    def _parse_number(text: str, patterns):
        import re
        for p in patterns:
            m = re.search(p, text)
            if m:
                try:
                    return float(m.group(1))
                except Exception:
                    return None
        return None

    def _extract_signals(text: str) -> dict:
        t = (text or "").lower()

        signals = {
            "setting": None,
            "wind_kmh": None,
            "temperature_c": None,
            "slope_deg": None,
            "humidity_pct": None,

            "terrain": {
                "mountain": any(w in t for w in ["mountain", "mountains", "montaña", "montañas"]),
                "steep": any(w in t for w in ["steep", "inclined", "slope", "inclinado", "pendiente"]),
                "rocky": any(w in t for w in ["rock", "rocks", "rocky", "piedra", "piedras", "rocoso"]),
                "urban_interface": any(w in t for w in ["wildland-urban", "wui", "interface", "interfaz"]),
            },

            "fuel": {
                "vegetation": any(w in t for w in ["vegetation", "veg", "brush", "shrubs", "trees", "bosque", "árbol", "arbol", "matorral"]),
                "dry": any(w in t for w in ["dry", "drought", "desert", "seco", "sequía", "sequia"]),
                "dense": any(w in t for w in ["dense", "thick", "muy", "mucha vegetación", "mucha vegetacion"]),
            },

            "exposure": {
                "people": any(w in t for w in ["people", "person", "humans", "civilians", "evac", "evacuation", "personas", "evacuar"]),
                "animals": any(w in t for w in ["animals", "wildlife", "livestock", "animales", "fauna", "ganado"]),
            },

            "infrastructure": {
                "electric": any(w in t for w in ["power", "powerline", "electric", "grid", "transformer", "substation", "eléctr", "electr"]),
                "buildings": any(w in t for w in ["building", "house", "home", "apartment", "factory", "industrial", "warehouse", "structure", "casa", "fábrica", "fabrica", "nave"]),
                "roads": any(w in t for w in ["road", "highway", "route", "carretera", "autopista"]),
                "water": any(w in t for w in ["water", "hydrant", "river", "lake", "reservoir", "agua", "hidrante", "río", "rio", "embalse"]),
            },
        }

        if any(w in t for w in ["forest", "wildland", "woods", "bosque"]):
            signals["setting"] = "forest"
        elif any(w in t for w in ["factory", "industrial", "plant", "warehouse", "fábrica", "fabrica", "nave"]):
            signals["setting"] = "industrial"
        elif any(w in t for w in ["house", "home", "apartment", "residential", "casa", "piso"]):
            signals["setting"] = "residential"
        elif any(w in t for w in ["city", "urban", "town", "ciudad", "urbano"]):
            signals["setting"] = "urban"

        signals["wind_kmh"] = _parse_number(
            t,
            [
                r"(\d+(?:\.\d+)?)\s*(?:km\/h|kmh)",
                r"wind\s*of\s*(\d+(?:\.\d+)?)",
                r"viento\s*de\s*(\d+(?:\.\d+)?)",
            ],
        )

        signals["temperature_c"] = _parse_number(
            t,
            [
                r"(\d+(?:\.\d+)?)\s*°\s*c",
                r"(\d+(?:\.\d+)?)\s*celsius",
                r"temperatura\s*(\d+(?:\.\d+)?)",
            ],
        )

        signals["humidity_pct"] = _parse_number(
            t,
            [
                r"(\d+(?:\.\d+)?)\s*%?\s*humidity",
                r"humedad\s*(\d+(?:\.\d+)?)",
            ],
        )

        signals["slope_deg"] = _parse_number(
            t,
            [
                r"(\d+(?:\.\d+)?)\s*°\s*slope",
                r"pendiente\s*(\d+(?:\.\d+)?)",
            ],
        )

        return signals

    def _roles(signals: dict) -> List[str]:
        roles: List[str] = []

        roles.append("thermal_barrier_refractory")
        roles.append("protective_coatings_refractory_family")

        if signals["infrastructure"]["electric"]:
            roles.append("electrical_insulation_near_infrastructure")

        if signals["infrastructure"]["buildings"] or signals["setting"] in {"residential", "industrial", "urban"}:
            roles.append("high_temp_structural_protection")

        w = signals.get("wind_kmh")
        if isinstance(w, (int, float)) and w >= 40:
            roles.append("high_wind_exposure_hardened_materials")

        if signals["fuel"]["dry"] or (signals.get("humidity_pct") is not None and signals["humidity_pct"] <= 25):
            roles.append("extended_heat_exposure_refractory")

        if signals["terrain"]["mountain"] or signals["terrain"]["steep"]:
            roles.append("mechanically_robust_transportable_materials")

        seen = set()
        out = []
        for r in roles:
            if r not in seen:
                out.append(r)
                seen.add(r)
        return out

    def _mp_proxy_criteria(roles: List[str], signals: dict) -> Tuple[dict, List[str]]:
        notes: List[str] = []

        criteria: dict = {
            "energy_above_hull": (0.0, 0.05),
        }
        notes.append("energy_above_hull <= 0.05 as stability proxy.")

        if any("electrical_insulation" in r for r in roles):
            criteria["band_gap"] = (3.0, 10.0)
            notes.append("band_gap >= 3.0 as insulation proxy.")

        criteria["elements"] = ["O"]
        notes.append("elements=['O'] to bias toward oxide refractories (broad).")

        return criteria, notes

    signals = _extract_signals(main_task_s)
    material_roles = _roles(signals)
    search_criteria, criteria_notes = _mp_proxy_criteria(material_roles, signals)

    plan = {
        "schema_version": "1.2",
        "created_at": datetime.utcnow().isoformat() + "Z",
        "main_task_improved": main_task_s,
        "guardrails": guardrails,
        "defaults": defaults,
        "fireground_signals": signals,
        "material_roles": material_roles,

        "retrieval": {
            "mode": "operational",
            "max_candidates": defaults["sample_number"],
            "page_size": 200,
            "timeout_s": 20,

            "search_criteria": search_criteria,
            "fields": defaults["fields_minimum"],
            "sample_number": defaults["sample_number"],
            "notes": criteria_notes,
        },

        "routing": {
            "after_planner": "AgentPlanReviewer",
            "on_approve": "AgentMaterialsRetriever",
            "on_revise": "AgentPlanner",
            "on_handoff": "Human",
        },
        "steps": [
            {
                "id": "retrieve",
                "agent": "AgentMaterialsRetriever",
                "purpose": "Execute Materials Project query using plan.retrieval.search_criteria and retrieve materials.",
                "inputs": ["retrieval.search_criteria", "retrieval.fields", "retrieval.sample_number"],
                "outputs": ["search_criteria", "fields", "sample_number", "mp_results"],
                "constraints": {
                    "whitelisted_search_keys_only": True,
                    "whitelisted_fields_only": True,
                    "sample_number_guardrails": sn,
                    "retriever_must_not_invent_filters": True,
                    "retriever_must_follow_plan_retrieval": True,
                },
            },
            {
                "id": "analyze",
                "agent": "AgentAnalyzer",
                "purpose": "Produce a data-grounded structured analysis from mp_results.",
                "inputs": ["mp_results", "main_task_improved"],
                "outputs": ["final_conclusion"],
                "constraints": {
                    "no_overclaiming": True,
                    "must_note_missing_fields": True,
                    "structured_json_output": True,
                },
            },
            {
                "id": "code",
                "agent": "AgentCoder",
                "purpose": "Generate reproducible artifacts (CSV/plots) from final_conclusion, with logs.",
                "inputs": ["final_conclusion"],
                "outputs": ["last_executed_code", "last_execution_output", "last_executed_file"],
                "constraints": {
                    "writes_to_project_folder": True,
                    "reproducible_outputs": True,
                },
            },
        ],
    }

    context_variables["plan"] = plan
    next_agent = plan["routing"]["after_planner"]
    context_variables["next_agent"] = next_agent

    return ReplyResult(
        message=json.dumps(plan, indent=2),
        target=AgentNameTarget(next_agent),
        context_variables=context_variables,
    )



# Tool Reviewer — review_plan

def review_plan_tool(
    decision: Annotated[
        Literal["approve", "revise", "handoff"],
        "approve -> go to retriever; revise -> go back to planner; handoff -> go back to human.",
    ],
    review_notes: Annotated[str, "Short, concrete notes about what is wrong/right in the plan."],
    context_variables: ContextVariables,
) -> ReplyResult:
    """Review context_variables['plan'] (multi-agent plan) and route to the next agent."""

    agent_name = "AgentPlanReviewer"
    plan = context_variables.get("plan")

    try:
        if plan is None or not isinstance(plan, dict):
            raise ValueError("Missing or invalid plan in context_variables['plan'].")
        if decision not in {"approve", "revise", "handoff"}:
            raise ValueError("decision must be one of: approve, revise, handoff.")
        if not isinstance(review_notes, str) or not review_notes.strip():
            raise ValueError("review_notes must be a non-empty string.")
    except ValueError as e:
        return ReplyResult(
            message=f"Error in review_plan_tool: {e}",
            target=AgentNameTarget(agent_name),
            context_variables=context_variables,
        )

    try:
        if not isinstance(plan.get("main_task_improved"), str) or not plan["main_task_improved"].strip():
            raise ValueError("plan.main_task_improved must be a non-empty string.")

        routing = plan.get("routing")
        if not isinstance(routing, dict):
            raise ValueError("plan.routing must be a dict.")

        for k in ("after_planner", "on_approve", "on_revise", "on_handoff"):
            if not isinstance(routing.get(k), str) or not routing[k].strip():
                raise ValueError(f"plan.routing.{k} must be a non-empty string.")

        steps = plan.get("steps")
        if not isinstance(steps, list) or len(steps) == 0:
            raise ValueError("plan.steps must be a non-empty list.")

        steps_by_id = {}
        for s in steps:
            if not isinstance(s, dict):
                raise ValueError("Each step in plan.steps must be a dict.")
            sid = s.get("id")
            if not isinstance(sid, str) or not sid.strip():
                raise ValueError("Each step must have a non-empty 'id'.")
            steps_by_id[sid] = s

        required_ids = {"retrieve", "analyze", "code"}
        missing = sorted(list(required_ids - set(steps_by_id.keys())))
        if missing:
            raise ValueError(f"Missing required step(s): {missing}")

        if steps_by_id["retrieve"].get("agent") != "AgentMaterialsRetriever":
            raise ValueError("Step 'retrieve' must use agent='AgentMaterialsRetriever'.")
        if steps_by_id["analyze"].get("agent") != "AgentAnalyzer":
            raise ValueError("Step 'analyze' must use agent='AgentAnalyzer'.")
        if steps_by_id["code"].get("agent") != "AgentCoder":
            raise ValueError("Step 'code' must use agent='AgentCoder'.")

        guardrails = plan.get("guardrails", {})
        if guardrails and not isinstance(guardrails, dict):
            raise ValueError("plan.guardrails must be a dict if present.")

        defaults = plan.get("defaults", {})
        if defaults and not isinstance(defaults, dict):
            raise ValueError("plan.defaults must be a dict if present.")

        sn_guard = None
        if isinstance(guardrails, dict):
            sn_guard = guardrails.get("sample_number")

        if sn_guard is not None:
            if not isinstance(sn_guard, dict):
                raise ValueError("plan.guardrails.sample_number must be a dict.")
            for kk in ("default", "min", "max"):
                if kk not in sn_guard:
                    raise ValueError(f"plan.guardrails.sample_number missing key '{kk}'.")
                if not isinstance(sn_guard[kk], int):
                    raise ValueError(f"plan.guardrails.sample_number.{kk} must be an int.")
            if sn_guard["min"] <= 0:
                raise ValueError("plan.guardrails.sample_number.min must be > 0.")
            if sn_guard["max"] < sn_guard["min"]:
                raise ValueError("plan.guardrails.sample_number.max must be >= min.")
            if not (sn_guard["min"] <= sn_guard["default"] <= sn_guard["max"]):
                raise ValueError("plan.guardrails.sample_number.default must be within [min, max].")

        if isinstance(defaults, dict) and "fields_minimum" in defaults:
            fm = defaults.get("fields_minimum")
            if not isinstance(fm, list) or not all(isinstance(x, str) and x.strip() for x in fm):
                raise ValueError("plan.defaults.fields_minimum must be a list of non-empty strings.")

    except ValueError as e:
        routing_safe = plan.get("routing") if isinstance(plan.get("routing"), dict) else {}
        next_agent = routing_safe.get("on_revise", "AgentPlanner")

        context_variables["review_status"] = "revise"
        context_variables["review_notes"] = f"{review_notes.strip()} | validation_error: {e}"
        context_variables["next_agent"] = next_agent

        return ReplyResult(
            message=f"Plan review -> revise\n{context_variables['review_notes']}",
            target=AgentNameTarget(next_agent),
            context_variables=context_variables,
        )

    context_variables["review_notes"] = review_notes.strip()

    if decision == "approve":
        next_agent = plan["routing"]["on_approve"]
        context_variables["review_status"] = "approved"
        context_variables["next_agent"] = next_agent
        return ReplyResult(
            message=f"Plan review -> approved\n{context_variables['review_notes']}",
            target=AgentNameTarget(next_agent),
            context_variables=context_variables,
        )

    if decision == "revise":
        next_agent = plan["routing"]["on_revise"]
        context_variables["review_status"] = "revise"
        context_variables["next_agent"] = next_agent
        return ReplyResult(
            message=f"Plan review -> revise\n{context_variables['review_notes']}",
            target=AgentNameTarget(next_agent),
            context_variables=context_variables,
        )

    next_agent = plan["routing"]["on_handoff"]
    context_variables["review_status"] = "handoff"
    context_variables["next_agent"] = next_agent
    return ReplyResult(
        message=f"Plan review -> handoff\n{context_variables['review_notes']}",
        target=AgentNameTarget(next_agent),
        context_variables=context_variables,
    )

# Tool — materials_project_retriever

def material_retriever(
    context_variables: ContextVariables,
    search_criteria: Optional[dict] = None,
    fields: Optional[List[str]] = None,
    sample_number: Optional[int] = None,
) -> ReplyResult:
    """Retrieve materials from Materials Project using plan.retrieval.search_criteria / fields / sample_number."""

    agent_name = "AgentMaterialsRetriever"

    plan = context_variables.get("plan", {})
    if not isinstance(plan, dict):
        plan = {}

    routing = plan.get("routing", {})
    if not isinstance(routing, dict):
        routing = {}
    next_on_revise = routing.get("on_revise", "AgentPlanner")

    retrieval = plan.get("retrieval", {})
    if not isinstance(retrieval, dict):
        retrieval = {}

    mode = retrieval.get("mode", "operational")
    max_candidates = retrieval.get("max_candidates", None)
    page_size = retrieval.get("page_size", 200)
    timeout_s = retrieval.get("timeout_s", 20)

    if mode not in {"operational", "exhaustive"}:
        mode = "operational"
    if not isinstance(page_size, int) or page_size <= 0:
        page_size = 200
    if not isinstance(timeout_s, (int, float)) or timeout_s <= 0:
        timeout_s = 20

    if search_criteria is None:
        search_criteria = retrieval.get("search_criteria", {})
    if fields is None:
        fields = retrieval.get("fields", [])
    if sample_number is None:
        sample_number = retrieval.get("sample_number", None)

    if sample_number is None:
        if isinstance(max_candidates, int) and max_candidates > 0:
            sample_number = max_candidates
        else:
            sample_number = 25

    try:
        if not isinstance(plan, dict) or not plan:
            raise ValueError("Missing or invalid plan in context_variables['plan'].")
        if not isinstance(retrieval, dict) or not retrieval:
            raise ValueError("Missing plan['retrieval'] dict.")
        if not isinstance(search_criteria, dict):
            raise ValueError("plan['retrieval']['search_criteria'] must be a dict.")
        if not isinstance(fields, list) or len(fields) == 0 or not all(isinstance(f, str) and f.strip() for f in fields):
            raise ValueError("plan['retrieval']['fields'] must be a non-empty list of non-empty strings.")
        if not isinstance(sample_number, int) or sample_number <= 0:
            raise ValueError("plan['retrieval']['sample_number'] must be a positive int.")
    except ValueError as e:
        context_variables["mp_results"] = []
        context_variables["search_criteria"] = {}
        context_variables["fields"] = []
        context_variables["sample_number"] = 0
        context_variables["next_agent"] = next_on_revise
        return ReplyResult(
            message=f"Error in material_retriever: {e}",
            target=AgentNameTarget(next_on_revise),
            context_variables=context_variables,
        )

    guardrails = plan.get("guardrails", {}) if isinstance(plan, dict) else {}
    sn_guard = guardrails.get("sample_number") if isinstance(guardrails, dict) else None

    if isinstance(sn_guard, dict):
        sn_min = sn_guard.get("min")
        sn_max = sn_guard.get("max")
        if isinstance(sn_min, int) and sample_number < sn_min:
            sample_number = sn_min
        if isinstance(sn_max, int) and sample_number > sn_max:
            sample_number = sn_max

    allowed_search_keys = {
        "band_gap",
        "energy_above_hull",
        "density",
        "num_sites",
        "k_voigt",
        "g_voigt",
        "elements",
        "chemsys",
        "exclude_elements",
        "excluded_elements",
    }
    allowed_fields = {
        "material_id",
        "formula_pretty",
        "band_gap",
        "energy_above_hull",
        "density",
        "volume",
        "symmetry",
        "nsites",
        "elements",
        "chemsys",
        "is_stable",
        "bulk_modulus",
        "shear_modulus",
    }

    try:
        if not all(isinstance(k, str) and k.strip() for k in search_criteria.keys()):
            raise ValueError("search_criteria keys must be non-empty strings.")

        bad_keys = [k for k in search_criteria.keys() if k not in allowed_search_keys]
        if bad_keys:
            raise ValueError(
                f"Unsupported search_criteria key(s): {bad_keys}. "
                f"Allowed keys: {sorted(list(allowed_search_keys))}"
            )

        bad_fields = [f for f in fields if f not in allowed_fields]
        if bad_fields:
            raise ValueError(
                f"Unsupported field(s): {bad_fields}. "
                f"Allowed fields: {sorted(list(allowed_fields))}"
            )
    except ValueError as e:
        context_variables["mp_results"] = []
        context_variables["search_criteria"] = search_criteria
        context_variables["fields"] = fields
        context_variables["sample_number"] = 0
        context_variables["next_agent"] = next_on_revise
        return ReplyResult(
            message=f"Error in material_retriever: {e}",
            target=AgentNameTarget(next_on_revise),
            context_variables=context_variables,
        )

    alias_map = {
        "num_sites": "nsites",
        "k_voigt": "bulk_modulus",
        "g_voigt": "shear_modulus",
        "pretty_formula": "formula_pretty",
        "excluded_elements": "exclude_elements",
    }

    def _is_range_tuple(v) -> bool:
        return isinstance(v, tuple) and len(v) == 2

    def _normalize_numeric_range(v):
        if _is_range_tuple(v):
            lo, hi = v
            if lo is None or hi is None:
                raise ValueError("Numeric range filters must be (min, max) with both values set.")
            if not isinstance(lo, (int, float)) or not isinstance(hi, (int, float)):
                raise ValueError("Numeric range filters must be (min, max) numbers.")
            if lo > hi:
                raise ValueError("Numeric range filters must satisfy min <= max.")
            return (lo, hi)

        if isinstance(v, (int, float)):
            return (v, v)

        return v

    normalized_criteria = {}
    try:
        for key, value in search_criteria.items():
            mapped_key = alias_map.get(key, key)

            if mapped_key in {"band_gap", "energy_above_hull", "density", "nsites", "bulk_modulus", "shear_modulus"}:
                normalized_criteria[mapped_key] = _normalize_numeric_range(value)
                continue

            if mapped_key in {"elements", "chemsys", "exclude_elements"}:
                if not (
                    isinstance(value, list)
                    and len(value) > 0
                    and all(isinstance(x, str) and x.strip() for x in value)
                ):
                    raise ValueError(f"{key} must be a non-empty list of non-empty strings.")
                normalized_criteria[mapped_key] = [x.strip() for x in value]
                continue

            normalized_criteria[mapped_key] = value
    except ValueError as e:
        context_variables["mp_results"] = []
        context_variables["search_criteria"] = search_criteria
        context_variables["fields"] = fields
        context_variables["sample_number"] = 0
        context_variables["next_agent"] = next_on_revise
        return ReplyResult(
            message=f"Error in material_retriever: {e}",
            target=AgentNameTarget(next_on_revise),
            context_variables=context_variables,
        )

    api_key = os.getenv("MP_API_KEY")
    if not api_key:
        context_variables["mp_results"] = []
        context_variables["search_criteria"] = search_criteria
        context_variables["fields"] = fields
        context_variables["sample_number"] = 0
        context_variables["next_agent"] = next_on_revise
        return ReplyResult(
            message="Error in material_retriever: Missing MP_API_KEY environment variable.",
            target=AgentNameTarget(next_on_revise),
            context_variables=context_variables,
        )

    try:
        with MPRester(api_key) as mpr:
            all_results = list(
                mpr.materials.summary.search(
                    fields=fields,
                    **normalized_criteria,
                )
            )
    except Exception as e:
        context_variables["mp_results"] = []
        context_variables["search_criteria"] = search_criteria
        context_variables["fields"] = fields
        context_variables["sample_number"] = 0
        context_variables["next_agent"] = next_on_revise
        return ReplyResult(
            message=f"Error in material_retriever while querying Materials Project: {e}",
            target=AgentNameTarget(next_on_revise),
            context_variables=context_variables,
        )

    if not all_results:
        context_variables["mp_results"] = []
        context_variables["search_criteria"] = search_criteria
        context_variables["fields"] = fields
        context_variables["sample_number"] = 0
        context_variables["next_agent"] = next_on_revise

        return ReplyResult(
            message="No materials found for the given search_criteria. Revise filters while keeping the main intent.",
            target=AgentNameTarget(next_on_revise),
            context_variables=context_variables,
        )

    if sample_number < len(all_results):
        results = random.sample(all_results, sample_number)
    else:
        results = all_results

    context_variables["search_criteria"] = search_criteria
    context_variables["fields"] = fields
    context_variables["sample_number"] = len(results)
    context_variables["mp_results"] = results
    context_variables["next_agent"] = "AgentAnalyzer"

    return ReplyResult(
        message=f"Retrieved {len(results)} materials from Materials Project with search_criteria={search_criteria}.",
        target=AgentNameTarget("AgentAnalyzer"),
        context_variables=context_variables,
    )



# Tool — final_conclusion_tool (Analyzer)

def final_conclusion_tool(
    context_variables: ContextVariables,
) -> ReplyResult:
    """Build a data-grounded structured analysis and store it in context_variables['final_conclusion']."""

    agent_name = "AgentAnalyzer"
    results = context_variables.get("mp_results", None)

    plan = context_variables.get("plan", {})
    routing = plan.get("routing", {}) if isinstance(plan, dict) else {}
    next_on_revise = routing.get("on_revise", "AgentPlanner")

    if results is None or results == []:
        context_variables["next_agent"] = next_on_revise
        return ReplyResult(
            message='{"error":"No materials available in context_variables[\\"mp_results\\"]."}',
            target=AgentNameTarget(next_on_revise),
            context_variables=context_variables,
        )

    if not isinstance(results, list):
        context_variables["next_agent"] = next_on_revise
        return ReplyResult(
            message='{"error":"context_variables[\\"mp_results\\"] must be a list."}',
            target=AgentNameTarget(next_on_revise),
            context_variables=context_variables,
        )

    # remove None entries
    results = [r for r in results if r is not None]
    if len(results) == 0:
        context_variables["next_agent"] = next_on_revise
        return ReplyResult(
            message='{"error":"context_variables[\\"mp_results\\"] contains no valid entries."}',
            target=AgentNameTarget(next_on_revise),
            context_variables=context_variables,
        )

    query_explanation = context_variables.get("main_task_improved", "")
    if not isinstance(query_explanation, str) or not query_explanation.strip():
        query_explanation = "Task description not available; analysis based only on retrieved mp_results."

    def _to_float(x):
        try:
            if x is None:
                return None
            return float(x)
        except Exception:
            return None

    def infer_application(band_gap):
        bg = _to_float(band_gap)
        if bg is None:
            return "Band gap not available or not numeric; cannot infer electronic/optical application domain."
        if bg < 0.1:
            return "Likely metallic or effectively gapless; candidate for conductive applications (verify is_metal if available)."
        if bg < 1.0:
            return "Narrow-gap semiconductor; IR/thermal-sensitive electronics possible depending on stability and structure."
        if bg < 3.0:
            return "Semiconductor regime; general electronics or optoelectronics possible depending on direct/indirect gap (not provided)."
        if bg <= 6.0:
            return "Wide-bandgap regime; candidates for power electronics, high-field devices, and UV optoelectronics depending on other properties."
        return "Very wide band gap; likely insulating behavior and potential deep-UV/insulating applications."

    def assess_stability(e_above_hull):
        eah = _to_float(e_above_hull)
        if eah is None:
            return "Energy above hull not available or not numeric; stability cannot be assessed from this dataset."
        if eah <= 1e-6:
            return "Stable (energy_above_hull ~ 0); higher likelihood of equilibrium synthesizability."
        if eah <= 0.05:
            return "Near-stable (<= 0.05 eV/atom); potentially synthesizable with suitable conditions."
        if eah <= 0.2:
            return "Metastable (0.05–0.2 eV/atom); synthesis may be nontrivial and condition-dependent."
        return "Likely unstable under equilibrium (> 0.2 eV/atom); lower prioritization unless strong motivation."

    def density_note(density):
        d = _to_float(density)
        if d is None:
            return "Density not available or not numeric; cannot provide qualitative structural/handling hints."
        if d < 2.5:
            return "Relatively low density; could be advantageous for weight-sensitive contexts (mechanical suitability unknown)."
        if d <= 6.0:
            return "Moderate density; typical for many ceramics/oxides/intermetallics (mechanical suitability unknown)."
        return "High density; may correlate with heavy-element content and higher mass per volume (mechanical suitability unknown)."

    def get_value(entry, k):
        if isinstance(entry, dict):
            return entry.get(k)
        return getattr(entry, k, None)

    materials = []
    summary = []

    for entry in results:
        material_id = get_value(entry, "material_id")
        formula_pretty = get_value(entry, "formula_pretty") or get_value(entry, "pretty_formula")

        band_gap = get_value(entry, "band_gap")
        density = get_value(entry, "density")
        volume = get_value(entry, "volume")
        e_above_hull = get_value(entry, "energy_above_hull") or get_value(entry, "e_above_hull")

        bg_f = _to_float(band_gap)
        eah_f = _to_float(e_above_hull)
        dens_f = _to_float(density)
        vol_f = _to_float(volume)

        limitations = []
        if bg_f is None:
            limitations.append("band_gap missing/not numeric -> application inference limited.")
        if eah_f is None:
            limitations.append("energy_above_hull missing/not numeric -> stability assessment limited.")
        if dens_f is None:
            limitations.append("density missing/not numeric -> limited physical/handling hints.")
        if vol_f is None:
            limitations.append("volume missing/not numeric -> no volumetric comparison possible.")
        limitations.append(
            "No direct/indirect gap, is_metal, elastic moduli, transport/thermal properties, or toxicity in retrieved fields -> avoid over-claiming."
        )

        mid = str(material_id) if material_id is not None else None
        fml = str(formula_pretty) if formula_pretty is not None else None

        materials.append(
            {
                "material_id": mid,
                "formula_pretty": fml,
                "key_properties": {
                    "band_gap": bg_f,
                    "energy_above_hull": eah_f,
                    "density": dens_f,
                    "volume": vol_f,
                },
                "stability_assessment": assess_stability(e_above_hull),
                "application_inference": infer_application(band_gap),
                "density_note": density_note(density),
                "limitations_and_unknowns": limitations,
            }
        )

        summary.append(
            {
                "material_id": mid,
                "formula_pretty": fml,
                "band_gap": bg_f,
                "energy_above_hull": eah_f,
                "density": dens_f,
            }
        )

    def score_candidate(m):
        eah = m["key_properties"].get("energy_above_hull")
        bg = m["key_properties"].get("band_gap")

        eah_v = eah if isinstance(eah, (int, float)) else 1e9
        bg_ok = isinstance(bg, (int, float))

        stability_score = -eah_v
        info_score = 0.0
        if bg_ok:
            info_score += 0.2
        if isinstance(eah, (int, float)):
            info_score += 0.2
        if isinstance(m["key_properties"].get("density"), (int, float)):
            info_score += 0.1

        return stability_score + info_score

    ranked = sorted(materials, key=score_candidate, reverse=True)
    best_candidates = [m["material_id"] for m in ranked[:2] if m.get("material_id") is not None]

    output = {
        "schema_version": "1.0",
        "created_at": datetime.utcnow().isoformat() + "Z",
        "query_explanation": query_explanation.strip(),
        "summary": summary,
        "materials": materials,
        "overall_recommendation": {
            "best_candidates": best_candidates,
            "rationale": [
                "Ranking prioritizes low energy_above_hull and availability of key fields.",
                "Application inference uses band_gap only; extend retrieval if you need is_metal, direct/indirect gap, or transport properties.",
            ],
        },
        "global_limitations": [
            "Missing properties are treated as unknown.",
            "No synthesis pathway or experimental validation is inferred from database metadata alone.",
        ],
    }

    context_variables["final_conclusion"] = output
    context_variables["next_agent"] = "AgentCoder"

    project_folder = os.environ.get("PROJECT_FOLDER", "ag2_project")
    os.makedirs(project_folder, exist_ok=True)

    with open(os.path.join(project_folder, "materials_data.json"), "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2, default=str, ensure_ascii=False)

    message = json.dumps(output, indent=2, default=str, ensure_ascii=False)

    return ReplyResult(
        message=message,
        target=AgentNameTarget("AgentCoder"),
        context_variables=context_variables,
    )



# Tool — python_coder

project_folder = os.path.abspath("ag2_project")
os.makedirs(project_folder, exist_ok=True)
os.environ["PROJECT_FOLDER"] = project_folder


class PythonCode(BaseModel):
    code: str = Field(..., description="Full python code executed by the coder agent")


def _safe_filename(name: str) -> str:
    name = os.path.basename(name.strip())
    if len(name) == 0:
        return "script.py"

    keep = []
    for ch in name:
        if ch.isalnum() or ch in {".", "_", "-"}:
            keep.append(ch)
    name = "".join(keep)

    if len(name) == 0:
        name = "script.py"
    if not name.endswith(".py"):
        name = name + ".py"
    if len(name) > 120:
        name = name[-120:]
    return name


def python_coder_tool(
    code: Annotated[str, "A single block of Python code to execute."],
    file_name: Annotated[str, "Name of the code file to store the executed script, e.g., 'script.py'"],
    context_variables: ContextVariables,
) -> ReplyResult:
    """Execute Python code, save it to ag2_project/<file_name>, update a persistent JSON context, and return the execution output."""

    agent_name = "AgentCoder"
    next_agent = "Human"

    try:
        if not isinstance(code, str) or len(code.strip()) == 0:
            raise ValueError("The 'code' parameter must be a non-empty string.")
        if not isinstance(file_name, str) or len(file_name.strip()) == 0:
            raise ValueError("The 'file_name' parameter must be a non-empty string.")
    except ValueError as e:
        return ReplyResult(
            message=f"Error in python_coder_tool: {e}",
            target=AgentNameTarget(agent_name),
            context_variables=context_variables,
        )

    safe_name = _safe_filename(file_name)

    runtime_prelude = (
        "import os\n"
        "PROJECT_FOLDER = os.environ.get('PROJECT_FOLDER', 'ag2_project')\n"
        "os.makedirs(PROJECT_FOLDER, exist_ok=True)\n"
    )

    full_code = "#!/usr/bin/env python3\n" + runtime_prelude + "\n" + code
    python_code_model = PythonCode(code=full_code)

    executor = LocalCommandLineCodeExecutor(timeout=600, work_dir=project_folder)
    code_block = CodeBlock(language="python", code=full_code)

    try:
        result = executor.execute_code_blocks([code_block])
    except Exception as e:
        message = "Code execution failed due to an internal executor error:\n" + f"{e}\n"
        return ReplyResult(
            message=message,
            target=AgentNameTarget(agent_name),
            context_variables=context_variables,
        )

    exit_code = result.exit_code
    output = result.output

    context_path = os.path.join(project_folder, "context_variables_data.json")

    try:
        with open(context_path, "r", encoding="utf-8") as f:
            context_file_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        context_file_data = {
            "execution_results": {},
            "execution_history": [],
            "execution_notes": [],
            "code": [],
            "code_error": "",
        }

    if exit_code != 0:
        context_file_data["code_error"] = python_code_model.model_dump()
    else:
        if "code" not in context_file_data or not isinstance(context_file_data.get("code"), list):
            context_file_data["code"] = []
        context_file_data["code"].append(python_code_model.model_dump())
        context_file_data["code_error"] = ""

    context_file_data["execution_results"][safe_name] = {
        "exit_code": exit_code,
        "output": output,
    }
    context_file_data["execution_history"].append(f"Executed {safe_name} with exit_code={exit_code}")
    context_file_data["execution_notes"].append("Executed code and stored output/exit_code.")

    code_output_path = os.path.join(project_folder, safe_name)
    os.makedirs(os.path.dirname(code_output_path), exist_ok=True)
    with open(code_output_path, "w", encoding="utf-8") as f:
        f.write(full_code)

    try:
        if getattr(result, "code_file", None) and os.path.exists(result.code_file):
            os.remove(result.code_file)
    except Exception:
        pass

    with open(context_path, "w", encoding="utf-8") as f:
        json.dump(context_file_data, f, indent=2, ensure_ascii=False)

    context_variables["last_executed_code"] = full_code
    context_variables["last_execution_output"] = output
    context_variables["last_executed_file"] = code_output_path

    if "execution_history" not in context_variables or not isinstance(context_variables.get("execution_history"), list):
        context_variables["execution_history"] = []
    context_variables["execution_history"].append(
        {"file_name": safe_name, "exit_code": exit_code}
    )

    if exit_code == 0:
        message = f"Code executed successfully.\n{output}"
    else:
        message = f"Code execution failed.\n{output}"

    context_variables["next_agent"] = next_agent

    return ReplyResult(
        message=message,
        target=AgentNameTarget(next_agent),
        context_variables=context_variables,
    )

# Tool — handoff_to_human

def handoff_to_human_tool(
    message: Annotated[str, "Short explanation of why the query cannot be handled by Materials Project."],
    context_variables: ContextVariables,
) -> ReplyResult:
    """Route the conversation back to the Human for query reformulation."""

    agent_name = "Human"

    try:
        if not isinstance(message, str) or not message.strip():
            raise ValueError("handoff message must be a non-empty string.")
    except ValueError as e:
        return ReplyResult(
            message=f"Error in handoff_to_human_tool: {e}",
            target=AgentNameTarget(agent_name),
            context_variables=context_variables,
        )

    context_variables["handoff_reason"] = message.strip()
    context_variables["next_agent"] = agent_name

    return ReplyResult(
        message=message.strip(),
        target=AgentNameTarget(agent_name),
        context_variables=context_variables,
    )
    
# Sanity check — tools loaded
_required_tools = [
    "task_improver_tool",
    "planner_tool",
    "review_plan_tool",
    "material_retriever",
    "final_conclusion_tool",
    "python_coder_tool",
    "handoff_to_human_tool",
]

_missing = [t for t in _required_tools if t not in globals() or not callable(globals()[t])]
if _missing:
    raise RuntimeError(f" Tools missing or not callable: {_missing}")

print(f"✓ Tools loaded OK ({len(_required_tools)}).")




✓ Tools loaded OK (7).


# 5. Manual tests for material_retriever


# 5. Manual tests for material_retriever 

print("Running manual tests for material_retriever...\n")

import os
import time
from autogen.agentchat.group import ContextVariables

def _make_ctx(search_criteria, fields, sample_number):
    plan = {
        "guardrails": {"sample_number": {"default": 10, "min": 1, "max": 10}},
        "routing": {"on_revise": "AgentPlanner"},
        "retrieval": {
            "search_criteria": search_criteria,
            "fields": fields,
            "sample_number": sample_number,
        },
    }
    return ContextVariables(data={"plan": plan})

if not os.getenv("MP_API_KEY"):
    raise RuntimeError("MP_API_KEY not set.")

def run_test(name, search_criteria, fields, sample_number):
    print(f"{name}...")
    t0 = time.time()

    ctx = _make_ctx(search_criteria, fields, sample_number)

    result = material_retriever(
        context_variables=ctx,
    )

    dt = time.time() - t0
    print(f"{name} done in {dt:.2f}s")
    print(result.message)
    if ctx.get("sample_number") is not None:
        print("effective sample_number:", ctx.get("sample_number"))
    print("-" * 60)


run_test(
    "Test 1",
    {
        "chemsys": ["Si-O"],
        "band_gap": (3, 5),
        "energy_above_hull": (0, 0.05),
    },
    ["material_id", "formula_pretty", "band_gap", "energy_above_hull"],
    5,
)

run_test(
    "Test 2",
    {
        "elements": ["Al", "O"],
        "band_gap": (0, 1),
        "energy_above_hull": (0, 0.05),
    },
    ["material_id", "band_gap", "energy_above_hull"],
    3,
)

run_test(
    "Test 3 (empty fields)",
    {
        "chemsys": ["Si-O"],
        "band_gap": (1, 3),
    },
    [],
    3,
)

run_test(
    "Test 4 (sample_number <= 0)",
    {
        "chemsys": ["Si-O"],
        "band_gap": (0, 10),
    },
    ["material_id"],
    0,
)

run_test(
    "Test 5 (invalid key)",
    {
        "invalid_property": (0, 1),
    },
    ["material_id"],
    3,
)

run_test(
    "Test 6 (sample_number clamp)",
    {
        "chemsys": ["Si-O"],
        "band_gap": (1, 4),
    },
    ["material_id", "band_gap", "energy_above_hull"],
    999,
)

print("\nManual tool tests completed.")



# 6. AGENTS (TaskImprover → Planner → PlanReviewer → MaterialsRetriever → Analyzer → Coder + Human)

In [5]:
# Agent: Task Improver

task_improver_message = """
You are AgentTaskImprover.

Your task is to improve the provided main task for its specific goal.

CRITERIA
- Analyze the consistency of the main task (does it make sense, is it complete, is it lacking information).
- Check if the main task is complete or if it lacks any information.
- Detect redundancy and remove it without changing the goal.
- If information is missing, make your best guess about what is missing and suggest a better formulation.
- Focus only on improving the task for the specific goal it sets.

AGENTS IN THE TEAM
- AgentPlanner: creates a plan JSON from main_task_improved.
- AgentPlanReviewer: validates the plan and routes approve/revise/handoff.
- AgentMaterialsRetriever: queries Materials Project using whitelisted keys/fields.
- AgentAnalyzer: produces structured JSON analysis from mp_results.
- AgentCoder: executes Python code to generate artifacts.
- Human: reformulates the query if needed.

TOOL USAGE
- You must always use task_improver_tool.
- Do not answer directly without using the tool.
"""

AgentTaskImprover = ConversableAgent(
    name="AgentTaskImprover",
    llm_config=llm_config,
    system_message=task_improver_message,
    human_input_mode="NEVER",
    functions=[task_improver_tool],
    function_map={"task_improver_tool": task_improver_tool},
)

# Agent: Planner

planner_message = """
You are AgentPlanner.

Your task is to create a clean multi-agent execution plan from context_variables["main_task_improved"].

RULES
- Use only information already present in context_variables.
- Keep the plan minimal and consistent with the toolchain.
- Build plan["retrieval"] with proxy Materials Project criteria (search_criteria, fields, sample_number).
- Always output the plan via planner_tool (JSON).

ROUTING
- After you build the plan, route to AgentPlanReviewer.

TOOL USAGE
- You must always use planner_tool.
- Do not answer directly without using the tool.
"""

AgentPlanner = ConversableAgent(
    name="AgentPlanner",
    llm_config=llm_config,
    system_message=planner_message,
    human_input_mode="NEVER",
    functions=[planner_tool],
    function_map={"planner_tool": planner_tool},
)

# Agent: Plan Reviewer

plan_reviewer_message = """
You are AgentPlanReviewer.

Your task is to review context_variables["plan"] and decide:
- approve -> go to retriever
- revise -> go back to planner
- handoff -> go back to human

RULES
- Validate the plan shape and basic wiring.
- Keep review_notes short and concrete.
- If the plan is missing required keys/steps, choose "revise".
- Do not change the plan here; only decide routing.

TOOL USAGE
- You must always use review_plan_tool.
- Do not answer directly without using the tool.
"""

AgentPlanReviewer = ConversableAgent(
    name="AgentPlanReviewer",
    llm_config=llm_config,
    system_message=plan_reviewer_message,
    human_input_mode="NEVER",
    functions=[review_plan_tool],
    function_map={"review_plan_tool": review_plan_tool},
)

# Agent: Materials Retriever

materials_retriever_message = """
You are AgentMaterialsRetriever.

Your task is to retrieve candidate materials from Materials Project
based strictly on the execution plan stored in context_variables["plan"].

WHAT YOU DO
- Read search_criteria, fields, and sample_number from plan["retrieval"].
- Call material_retriever exactly once.
- Do not invent, modify, or infer filters or fields.
- Store results in context_variables["mp_results"].
- Do not analyze or rank materials.

RULES
- Use only whitelisted search_criteria keys and fields.
- Do not invent semantic filters.
- Follow sample_number guardrails from the plan.
- If no materials are found, route according to plan.routing.

SEARCH_CRITERIA KEYS (WHITELIST)
- band_gap
- energy_above_hull
- density
- num_sites
- k_voigt
- g_voigt
- elements
- chemsys
- exclude_elements
- excluded_elements

FIELDS (WHITELIST)
- material_id
- formula_pretty
- band_gap
- energy_above_hull
- density
- volume
- symmetry
- nsites
- elements
- chemsys
- is_stable
- bulk_modulus
- shear_modulus

FORMAT RULES
- Numeric filters use tuples: (min, max)
- Use None for open-ended bounds
- List filters must be lists of strings
- No Mongo-style operators

TOOL USAGE
- Always call material_retriever with no arguments.
- material_retriever must read search_criteria, fields, and sample_number exclusively from context_variables["plan"]["retrieval"].
- Do not pass search_criteria, fields, or sample_number explicitly.
- Do not answer directly.
"""

AgentMaterialsRetriever = ConversableAgent(
    name="AgentMaterialsRetriever",
    llm_config=llm_config,
    system_message=materials_retriever_message,
    human_input_mode="NEVER",
    functions=[material_retriever],
    function_map={"material_retriever": material_retriever},
)

# Agent: Analyzer

analyzer_message = """
You are AgentAnalyzer.

ROLE
- Build a data-grounded conclusion from Materials Project results.

INPUTS (from context_variables)
- context_variables["mp_results"]: list of entries returned by the retriever tool.
- context_variables.get("main_task_improved", ""): task intent.
- If main_task_improved is missing, proceed using mp_results only.

YOUR TASKS
1) Read mp_results and use only available fields (no assumptions).
2) Produce analysis derived from the data:
   - band_gap -> application inference (qualitative)
   - energy_above_hull -> stability assessment (qualitative)
   - density -> qualitative note only (no mechanical claims)
3) Output must be JSON only (no prose outside JSON).
4) Store the JSON in context_variables["final_conclusion"].

OUTPUT FORMAT
- You must return the JSON produced by final_conclusion_tool.
- The JSON may include metadata keys like schema_version/created_at/global_limitations if present.

TOOL USAGE
- Always call final_conclusion_tool.
- Do not answer directly.
"""

AgentAnalyzer = ConversableAgent(
    name="AgentAnalyzer",
    llm_config=llm_config,
    system_message=analyzer_message,
    human_input_mode="NEVER",
    functions=[final_conclusion_tool],
    function_map={"final_conclusion_tool": final_conclusion_tool},
)

# Agent: Coder

coder_message = """
You are AgentCoder.

ROLE
- Generate and execute Python code when needed by the pipeline.

TASKS
- When you are invoked by the pipeline and context_variables contains mp_results, you must generate at least ONE computational artifact automatically:
  - either export a CSV summary table (e.g., material_id, formula_pretty, band_gap, energy_above_hull, density), and/or
  - plot a histogram of band_gap distribution.
- The Analyzer persists the structured results to materials_data.json in the project folder. Use that file as the single source of truth for plots/tables.
- Assume python_coder_tool sets the working directory to PROJECT_FOLDER (ag2_project). Therefore, use relative paths only:
  - read "materials_data.json" (do not prefix with "ag2_project/")
  - write outputs like "summary_table.csv" and "band_gap_histogram.png" (no "ag2_project/" prefix)
- Prefer generating both CSV and histogram in a single python_coder_tool call unless there is a strong reason to split.
- Execute all code only via python_coder_tool.
- Save executed code under ag2_project/<file_name>.
- Use the tool output to decide the next step.

TOOL USAGE
- Always call python_coder_tool for any code execution.
- Do not execute code directly.
- Do not answer with analysis only; the response must come from the tool output.
"""

AgentCoder = ConversableAgent(
    name="AgentCoder",
    llm_config=llm_config,
    system_message=coder_message,
    human_input_mode="NEVER",
    functions=[python_coder_tool],
    function_map={"python_coder_tool": python_coder_tool},
)

# Human Agent

Human = ConversableAgent(
    name="Human",
    llm_config=None,
    human_input_mode="ALWAYS",
)
print("✓ Agents created.")

✓ Agents created.


# 7. Pattern definition 


In [6]:
# 7. Pattern definition

pattern = DefaultPattern(
    initial_agent=AgentTaskImprover,
    user_agent=Human,
    agents=[
        AgentTaskImprover,
        AgentPlanner,
        AgentPlanReviewer,
        AgentMaterialsRetriever,
        AgentAnalyzer,
        AgentCoder,
    ],
    context_variables=context_variables,
)


# 8. Group Chat

In [7]:
print("Hi! Type your query and press Enter.\n")
human_query = input("Query: ").strip()

result, context, _ = initiate_group_chat(
    pattern=pattern,
    messages=[{"role": "user", "content": human_query}],
    max_rounds=20,
)



Hi! Type your query and press Enter.



Query:  Design a materials shortlist for a wildfire field-deployable thermal barrier and protective coating system for ground crews operating under sustained 60 km/h wind. Prioritize: (1) high thermal stability / refractory behavior, (2) chemical inertness in oxidizing environments, (3) electrical insulation to reduce ignition risk near power infrastructure, (4) low toxicity / avoid heavy metals. Return candidate oxide/ceramic materials and justify using only Materials Project retrievable proxies (e.g., energy_above_hull, band_gap, density, symmetry, elements, is_stable)


Human (to chat_manager):

Design a materials shortlist for a wildfire field-deployable thermal barrier and protective coating system for ground crews operating under sustained 60 km/h wind. Prioritize: (1) high thermal stability / refractory behavior, (2) chemical inertness in oxidizing environments, (3) electrical insulation to reduce ignition risk near power infrastructure, (4) low toxicity / avoid heavy metals. Return candidate oxide/ceramic materials and justify using only Materials Project retrievable proxies (e.g., energy_above_hull, band_gap, density, symmetry, elements, is_stable)

--------------------------------------------------------------------------------

Next speaker: AgentTaskImprover

AgentTaskImprover (to chat_manager):

***** Suggested tool call (call_DBTkycMd0T7pkEgzS0UMcyhe): task_improver_tool *****
Arguments: 
{"analysis":"The task is generally well-stated with clear priorities and criteria for selection. However, there are some areas that could be clarified or 

/var/folders/wn/mpnj0v3s1_1fd88s9l4c9pth0000gn/T/ipykernel_94169/4144494903.py:263: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_at": datetime.utcnow().isoformat() + "Z",


AgentPlanReviewer (to chat_manager):

***** Suggested tool call (call_ktEziHXnvNnjgu9WU4OvIaFJ): review_plan_tool *****
Arguments: 
{"decision":"revise","review_notes":"The plan is missing clear criteria for selecting specific materials. It also lacks detailed evaluation points for electrical insulation and a strong focus on chemical inertness. Define specific benchmarks for 'band_gap' and explain how 'elements' will be used to assess chemical inertness. Ensure output notes how chosen materials align with specific criteria."}
*********************************************************************************

--------------------------------------------------------------------------------

Next speaker: _Group_Tool_Executor


>>>>>>>> EXECUTING FUNCTION review_plan_tool...
Call ID: call_ktEziHXnvNnjgu9WU4OvIaFJ
Input arguments: {'decision': 'revise', 'review_notes': "The plan is missing clear criteria for selecting specific materials. It also lacks detailed evaluation points for electric

Retrieving SummaryDoc documents:   0%|          | 0/11828 [00:00<?, ?it/s]


>>>>>>>> EXECUTED FUNCTION material_retriever...
Call ID: call_2YEsmL7QhHv4LWcm5wTZaSdr
Input arguments: {}
Output:
Error in material_retriever while querying Materials Project: HTTPSConnectionPool(host='api.materialsproject.org', port=443): Max retries exceeded with url: /materials/summary/?band_gap_min=3.0&band_gap_max=10.0&energy_above_hull_min=0.0&energy_above_hull_max=0.05&elements=O&_limit=1000&_fields=material_id%2Cformula_pretty%2Cband_gap%2Cenergy_above_hull%2Cdensity%2Cvolume%2Csymmetry%2Cnsites%2Celements%2Cchemsys%2Cis_stable&_skip=5000 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='api.materialsproject.org', port=443): Read timed out. (read timeout=20)"))
***** ReplyResult transition (AgentMaterialsRetriever): AgentPlanner *****
_Group_Tool_Executor (to chat_manager):

***** Response from calling tool (call_2YEsmL7QhHv4LWcm5wTZaSdr) *****
Error in material_retriever while querying Materials Project: HTTPSConnectionPool(host='api.materialsproject.org', port=443): M